In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import re
import os
import datetime

# Function to send email
def send_email(recipient_email, subject, body):
    # Retrieve email credentials from environment variables
    from_email = os.getenv("EMAIL_USER")
    password = os.getenv("EMAIL_PASS")
    
    # Check if credentials are loaded correctly
    if not from_email or not password:
        raise ValueError("Email credentials not found. Make sure to set the EMAIL_USER and EMAIL_PASS environment variables.")
    
    # Set up the MIME
    message = MIMEMultipart()
    message["From"] = from_email
    message["To"] = recipient_email
    message["Subject"] = subject
    
    # Add the body to the email
    message.attach(MIMEText(body, "plain"))
    
    # Send the email
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(from_email, password)
            server.sendmail(from_email, recipient_email, message.as_string())
        print(f"Email sent to {recipient_email}")
    except Exception as e:
        print(f"Failed to send email to {recipient_email}. Error: {e}")
        
# Function to compose the email body
def compose_email_body(masked_info, jd_score, final_score, ai_experience):
    # Start building the email content
    body = f"Dear {masked_info['full_name']},\n\n"
    
    body += f"Thank you for submitting your resume. Here's a detailed breakdown of your CV:\n\n"
    
    # CV Score
    body += f"🔹 **Overall Resume Score**: {final_score['scores']['overall']}/100\n\n"
    
    # Breakdown of the score components
    body += f"**Breakdown of Score Components**:\n"
    body += f"📋 **Formatting**: {final_score['scores']['formatting']}/20\n"
    body += f"💼 **Experience**: {final_score['scores']['experience']}/25\n"
    body += f"🎓 **Education**: {final_score['scores']['education']}/20\n"
    body += f"💻 **Skills**: {final_score['scores']['skills']}/25\n\n"
    
    # AI Experience (if applicable)
    if ai_experience["has_ai_experience"]:
        body += f"🤖 **AI Experience**: You have experience in the following areas: {', '.join(ai_experience['ai_skills'])}\n\n"
    else:
        body += f"🤖 **AI Experience**: No notable AI experience found.\n\n"
    
    # Notable feedback (strengths/weaknesses)
    if final_score['scores']['overall'] < 50:
        body += "⚠️ **Feedback**: Your CV needs improvements in terms of formatting, experience, and skills. Consider enhancing your technical skills and ensuring your resume has a clear structure.\n\n"
    elif final_score['scores']['overall'] < 70:
        body += "⚠️ **Feedback**: Your CV is decent, but there’s room for improvement in formatting and showcasing experience. Consider highlighting key accomplishments more clearly.\n\n"
    else:
        body += "🎉 **Feedback**: Excellent CV! You’ve demonstrated strong skills and experience, especially in AI and programming. Keep up the great work!\n\n"
  # Optional: Next steps or encouragement
    body += "🌟 **Next Steps**: We encourage you to keep improving your skills, particularly in the areas of machine learning and AI. Best of luck in your job search!\n\n"
    
    body += "Best regards,\nYour Resume Review Team"
    
    return body